In [1]:
import pandas as pd
import numpy as np
import random
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier

In [2]:
#General Functions

def featureNormalize(X): 
    return (X - np.mean(X, axis = 0)) / np.std(X, axis = 0, ddof = 1)

In [3]:
#Linear Regression Functions

def computeCostLinear(yhat, y):
    # (1/2m) * Σ[i->1:m] (Xⁱθ - yⁱ)²
    # Vector Implementation: (1/2m) * (Xθ - y)ᵀ.(Xθ - y)
    m = len(y)
    return (1.0 / (2 * m)) * np.dot((yhat - y).transpose(), yhat - y)[0][0]


def gradientDescentLinear(X, y, theta, alpha, threshold, max_iter = 1e+4):
    # As long as percent change in train error is greater than threshold
    # θⱼ := θⱼ - α * (1/m) * Σ[i->1:m] (Xⁱθ - yⁱ)Xⱼⁱ for j->0:n
    # Vector Implementation: θ := θ - α * (1/m) * (Xᵀ.(X.θ - y))
    m = len(y)
    J_theta = []
    J_theta.append({'iter' : 0, 'J_hist' : computeCostLinear(np.dot(X, theta), y), 'theta' : theta})
    pct_change = 100
    i = 0
    while pct_change > threshold and i < max_iter:
        delta = (1.0 / m) * np.dot(X.transpose(), np.dot(X, theta) - y)
        theta = theta - (alpha * delta)
        J_theta.append({'iter' : i + 1, 'J_hist' : computeCostLinear(np.dot(X, theta), y), 'theta' : theta})
        i += 1
        pct_change = ((J_theta[-2]['J_hist'] - J_theta[-1]['J_hist']) / J_theta[-2]['J_hist']) * 100
    return J_theta


def rmse(pred, real):
    # √(1/m) * Σ[i->1:m] (predⁱ - realⁱ)²
    # Vector Implementation: √(1/m) * (pred - real)ᵀ.(pred - real)
    m = len(real)
    return np.sqrt((1.0 / m) * np.dot((pred - real).transpose(), pred - real)[0][0])

In [4]:
#Preprocessing

df_bike = pd.read_csv('C:/Users/adity/SeoulBikeData.csv', encoding='latin-1')

df_bike.columns = ['date', 'bike_count', 'hour', 'temperature', 'humidity', 'wind_speed', 'visibility', 'dew_point_temp', 
                   'solar_radiation', 'rainfall', 'snowfall', 'season', 'holiday', 'functioning_day']

df_bike['date'] = pd.to_datetime(df_bike['date'], format = '%d/%m/%Y')
df_bike['hour'] = df_bike['hour'].astype('category')
df_bike['season'] = df_bike['season'].astype('category')
df_bike['holiday'] = df_bike['holiday'].astype('category')
df_bike['functioning_day'] = df_bike['functioning_day'].astype('category')
df_bike['day'] = df_bike['date'].dt.day_name().astype('category')

df_bike = df_bike[df_bike['functioning_day'] == 'Yes']
df_bike = df_bike.drop(['date', 'functioning_day', 'dew_point_temp'], axis = 1)
df_bike = df_bike.reset_index(drop = True)

df_bike

,bike_count,hour,temperature,humidity,wind_speed,visibility,solar_radiation,rainfall,snowfall,season,holiday,day
0,254,0,-5.2,37,2.2,2000,0.0,0.0,0.0,Winter,No Holiday,Friday
1,204,1,-5.5,38,0.8,2000,0.0,0.0,0.0,Winter,No Holiday,Friday
2,173,2,-6.0,39,1.0,2000,0.0,0.0,0.0,Winter,No Holiday,Friday
3,107,3,-6.2,40,0.9,2000,0.0,0.0,0.0,Winter,No Holiday,Friday
4,78,4,-6.0,36,2.3,2000,0.0,0.0,0.0,Winter,No Holiday,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...
8460,1003,19,4.2,34,2.6,1894,0.0,0.0,0.0,Autumn,No Holiday,Friday
8461,764,20,3.4,37,2.3,2000,0.0,0.0,0.0,Autumn,No Holiday,Friday
8462,694,21,2.6,39,0.3,1968,0.0,0.0,0.0,Autumn,No Holiday,Friday
8463,712,22,2.1,41,1.0,1859,0.0,0.0,0.0,Autumn,No Holiday,Friday


In [5]:
#All Features

df_full = df_bike

ddf_full = pd.get_dummies(df_full['hour'], drop_first = True, prefix = 'hour:')
ddf_full = pd.concat([ddf_full, pd.get_dummies(df_full['day'], drop_first = True, prefix = 'day:')], axis = 1)
ddf_full = pd.concat([ddf_full, pd.get_dummies(df_full['season'], drop_first = True, prefix = 'season:')], axis = 1)
ddf_full = pd.concat([ddf_full, pd.get_dummies(df_full['holiday'],  prefix = 'holiday:')], axis = 1)
ddf_full = ddf_full.drop('holiday:_No Holiday', axis = 1)

df_full = df_full.drop(['hour', 'day', 'season', 'holiday'], axis = 1)

y = pd.DataFrame(df_full['bike_count']).reset_index(drop = True)

df_full = df_full.drop('bike_count', axis = 1)

Xf = pd.concat([featureNormalize(df_full), ddf_full], axis = 1).reset_index(drop = True)
Xf.insert(loc = 0, column = 'intercept', value = 1.0)

Xf

,intercept,temperature,humidity,wind_speed,visibility,solar_radiation,rainfall,snowfall,hour:_1,hour:_2,...,day:_Monday,day:_Saturday,day:_Sunday,day:_Thursday,day:_Tuesday,day:_Wednesday,season:_Spring,season:_Summer,season:_Winter,holiday:_Holiday
0,1.0,-1.484675,-1.032334,0.458402,0.929522,-0.654041,-0.132487,-0.17494,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1.0,-1.509459,-0.983517,-0.895195,0.929522,-0.654041,-0.132487,-0.17494,1,0,...,0,0,0,0,0,0,0,0,1,0
2,1.0,-1.550766,-0.934701,-0.701824,0.929522,-0.654041,-0.132487,-0.17494,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1.0,-1.567289,-0.885884,-0.798509,0.929522,-0.654041,-0.132487,-0.17494,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1.0,-1.550766,-1.081151,0.555088,0.929522,-0.654041,-0.132487,-0.17494,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8460,1.0,-0.708096,-1.178784,0.845144,0.755481,-0.654041,-0.132487,-0.17494,0,0,...,0,0,0,0,0,0,0,0,0,0
8461,1.0,-0.774188,-1.032334,0.555088,0.929522,-0.654041,-0.132487,-0.17494,0,0,...,0,0,0,0,0,0,0,0,0,0
8462,1.0,-0.840279,-0.934701,-1.378622,0.876981,-0.654041,-0.132487,-0.17494,0,0,...,0,0,0,0,0,0,0,0,0,0
8463,1.0,-0.881587,-0.837068,-0.701824,0.698014,-0.654041,-0.132487,-0.17494,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
Xf_train = Xf.sample(random_state = 42, frac = 0.7).sort_index()
y_train = y[Xf.index.isin(Xf_train.index)]
Xf_test = Xf[~Xf.index.isin(Xf_train.index)]
y_test = y[~Xf.index.isin(Xf_train.index)]

c_train =  y_train > np.median(y)
c_test = y_test > np.median(y)

Xf_train.shape, y_train.shape, c_train.shape, Xf_test.shape, y_test.shape, c_test.shape

((5926, 41), (5926, 1), (5926, 1), (2539, 41), (2539, 1), (2539, 1))

In [7]:
#Randomly Selected Features

df_random = df_bike.drop(['bike_count'], axis = 1)

random.seed(42)
var_list_random = random.sample(list(df_random.columns), 8)
print(var_list_random)
# ['day', 'temperature', 'hour', 'visibility', 'holiday', 'rainfall', 'solar_radiation', 'season']

df_random = df_random[var_list_random]
ddf_random = pd.get_dummies(df_random['hour'], drop_first = True, prefix = 'hour:')
ddf_random = pd.concat([ddf_random, pd.get_dummies(df_random['day'], drop_first = True, prefix = 'day:')], axis = 1)
ddf_random = pd.concat([ddf_random, pd.get_dummies(df_random['season'], drop_first = True, prefix = 'season:')], axis = 1)
ddf_random = pd.concat([ddf_random, pd.get_dummies(df_random['holiday'],  prefix = 'holiday:')], axis = 1)
ddf_random = ddf_random.drop('holiday:_No Holiday', axis = 1)

df_random = df_random.drop(['hour', 'day', 'season', 'holiday'], axis = 1)

Xr = pd.concat([featureNormalize(df_random), ddf_random], axis = 1).reset_index(drop = True)
Xr.insert(loc = 0, column = 'intercept', value = 1.0)

Xr

['day', 'temperature', 'hour', 'visibility', 'holiday', 'rainfall', 'solar_radiation', 'season']


,intercept,temperature,visibility,rainfall,solar_radiation,hour:_1,hour:_2,hour:_3,hour:_4,hour:_5,...,day:_Monday,day:_Saturday,day:_Sunday,day:_Thursday,day:_Tuesday,day:_Wednesday,season:_Spring,season:_Summer,season:_Winter,holiday:_Holiday
0,1.0,-1.484675,0.929522,-0.132487,-0.654041,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1.0,-1.509459,0.929522,-0.132487,-0.654041,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1.0,-1.550766,0.929522,-0.132487,-0.654041,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1.0,-1.567289,0.929522,-0.132487,-0.654041,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1.0,-1.550766,0.929522,-0.132487,-0.654041,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8460,1.0,-0.708096,0.755481,-0.132487,-0.654041,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8461,1.0,-0.774188,0.929522,-0.132487,-0.654041,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8462,1.0,-0.840279,0.876981,-0.132487,-0.654041,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8463,1.0,-0.881587,0.698014,-0.132487,-0.654041,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
Xr_train = Xr[Xr.index.isin(Xf_train.index)]
Xr_test = Xr[~Xr.index.isin(Xf_train.index)]

Xr_train.shape, y_train.shape, c_train.shape, Xr_test.shape, y_test.shape, c_test.shape

((5926, 38), (5926, 1), (5926, 1), (2539, 38), (2539, 1), (2539, 1))

In [9]:
#Important Features

rf = RandomForestClassifier(random_state=42) 
rf.fit(Xf_train, y_train.values.ravel()) 

pd.DataFrame({'feature' : [i.split(':', 1)[0] for i in Xf_train.columns], 
              'importance' : rf.feature_importances_}).groupby('feature').sum().sort_values('importance', ascending = False)

,importance
feature,
temperature,0.154672
humidity,0.145239
wind_speed,0.141602
hour,0.140036
visibility,0.131811
day,0.127796
solar_radiation,0.079875
season,0.045137
rainfall,0.011641


In [10]:
var_list_top = ['temperature', 'humidity', 'wind_speed', 'hour', 'visibility', 'day', 'solar_radiation', 'season']

df_top = df_bike[var_list_top]
ddf_top = pd.get_dummies(df_top['hour'], drop_first = True, prefix = 'hour:')
ddf_top = pd.concat([ddf_top, pd.get_dummies(df_top['day'], drop_first = True, prefix = 'day:')], axis = 1)
ddf_top = pd.concat([ddf_top, pd.get_dummies(df_top['season'], drop_first = True, prefix = 'season:')], axis = 1)

df_top = df_top.drop(['hour', 'day', 'season'], axis = 1)

Xt = pd.concat([featureNormalize(df_top), ddf_top], axis = 1).reset_index(drop = True)
Xt.insert(loc = 0, column = 'intercept', value = 1.0)

Xt

,intercept,temperature,humidity,wind_speed,visibility,solar_radiation,hour:_1,hour:_2,hour:_3,hour:_4,...,hour:_23,day:_Monday,day:_Saturday,day:_Sunday,day:_Thursday,day:_Tuesday,day:_Wednesday,season:_Spring,season:_Summer,season:_Winter
0,1.0,-1.484675,-1.032334,0.458402,0.929522,-0.654041,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1.0,-1.509459,-0.983517,-0.895195,0.929522,-0.654041,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1.0,-1.550766,-0.934701,-0.701824,0.929522,-0.654041,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1.0,-1.567289,-0.885884,-0.798509,0.929522,-0.654041,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,1.0,-1.550766,-1.081151,0.555088,0.929522,-0.654041,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8460,1.0,-0.708096,-1.178784,0.845144,0.755481,-0.654041,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8461,1.0,-0.774188,-1.032334,0.555088,0.929522,-0.654041,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8462,1.0,-0.840279,-0.934701,-1.378622,0.876981,-0.654041,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8463,1.0,-0.881587,-0.837068,-0.701824,0.698014,-0.654041,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
Xt_train = Xt[Xt.index.isin(Xf_train.index)]
Xt_test = Xt[~Xt.index.isin(Xf_train.index)]

Xt_train.shape, y_train.shape, c_train.shape, Xt_test.shape, y_test.shape, c_test.shape

((5926, 38), (5926, 1), (5926, 1), (2539, 38), (2539, 1), (2539, 1))

In [ ]:
#Linear Regression Models
#Learning Rate vs Train and Test RMSE

threshold = 1e-3
alphas = [1e+2, 3e+1, 1e+1, 3e+0, 1e+0, 3e-1, 1e-1, 3e-2, 1e-2, 3e-3, 1e-3, 3e-4, 1e-4, 3e-5, 1e-5]

lin_alpha_errors = []
for alpha in alphas:
    lin_alpha_error = {}
    theta = np.zeros((Xf_train.shape[1], 1), dtype = float)
    J_theta = gradientDescentLinear(Xf_train.values, y_train.values, theta, alpha, threshold)
    lin_alpha_error['learning_rate'] = alpha
    lin_alpha_error['converging_iteration'] = J_theta[-1]['iter']
    lin_alpha_error['train_rmse'] = rmse(np.dot(Xf_train.values, J_theta[-1]['theta']), y_train.values)
    lin_alpha_error['test_rmse'] = rmse(np.dot(Xf_test.values, J_theta[-1]['theta']), y_test.values)
    lin_alpha_errors.append(lin_alpha_error)
    
df_lin_alpha_errors = pd.DataFrame(lin_alpha_errors)
df_lin_alpha_errors

In [52]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = df_lin_alpha_errors['learning_rate'], y = df_lin_alpha_errors['train_rmse'], 
                         mode = 'lines+markers', name = 'Train RMSE'))
fig.add_trace(go.Scatter(x = df_lin_alpha_errors['learning_rate'], y = df_lin_alpha_errors['test_rmse'], 
                         mode = 'lines+markers', name = 'Test RMSE'))
fig.update_xaxes(type = 'log')
fig.update_yaxes(type = 'log')
fig.update_layout(title = 'Learning Rate vs Train and Test RMSE', 
                  xaxis_title = 'Learning Rate', yaxis_title = 'RMSE',
                  hovermode = 'x unified')

fig.show()

In [53]:
#Threshold for Convergence vs Train and Test RMSE

alpha = 3e-1
thresholds = [1e+1, 1e+0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]

lin_thresh_errors = []
for threshold in thresholds:
    lin_thresh_error = {}
    theta = np.zeros((Xf_train.shape[1], 1), dtype = float)
    J_theta = gradientDescentLinear(Xf_train.values, y_train.values, theta, alpha, threshold)
    lin_thresh_error['threshold'] = threshold
    lin_thresh_error['converging_iteration'] = J_theta[-1]['iter']
    lin_thresh_error['train_rmse'] = rmse(np.dot(Xf_train.values, J_theta[-1]['theta']), y_train.values)
    lin_thresh_error['test_rmse'] = rmse(np.dot(Xf_test.values, J_theta[-1]['theta']), y_test.values)
    lin_thresh_errors.append(lin_thresh_error)
    
df_lin_thresh_errors = pd.DataFrame(lin_thresh_errors)
df_lin_thresh_errors

,threshold,converging_iteration,train_rmse,test_rmse
0,1.000000e+01,4,516.386797,514.678308
1,1.000000e+00,17,462.624832,457.272787
2,1.000000e-01,123,384.436489,376.422058
3,1.000000e-02,254,375.261342,367.275887
4,1.000000e-03,430,374.054212,366.233839
5,1.000000e-04,659,373.895789,366.161168
6,1.000000e-05,928,373.876768,366.165006
7,1.000000e-06,1249,373.874555,366.168979
8,1.000000e-07,2454,373.873980,366.174262
9,1.000000e-08,4923,373.873800,366.178287


In [54]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = df_lin_thresh_errors['threshold'], y = df_lin_thresh_errors['train_rmse'], 
                         mode = 'lines+markers', name = 'Train RMSE'))
fig.add_trace(go.Scatter(x = df_lin_thresh_errors['threshold'], y = df_lin_thresh_errors['test_rmse'], 
                         mode = 'lines+markers', name = 'Test RMSE'))
fig.update_xaxes(type = 'log')
fig.update_layout(title = 'Threshold for Convergence vs Train and Test RMSE (Learning Rate = 0.3)', 
                  xaxis_title = 'Threshold', yaxis_title = 'RMSE',
                  hovermode = 'x unified')

fig.show()

In [55]:
#Train and Test RMSE at various Iterations for selected Learning Rate and Threshold for Convergence

alpha = 3e-1
threshold = 1e-4

theta = np.zeros((Xf_train.shape[1], 1), dtype = float)
J_theta = gradientDescentLinear(Xf_train.values, y_train.values, theta, alpha, threshold)

lin_select_theta = J_theta[-1]['theta']

lin_param_errors = []
for i in J_theta:
    lin_param_error = {}
    lin_param_error['iteration'] = i['iter']
    lin_param_error['train_rmse'] = rmse(np.dot(Xf_train.values, i['theta']), y_train.values)
    lin_param_error['test_rmse'] = rmse(np.dot(Xf_test.values, i['theta']), y_test.values)
    lin_param_errors.append(lin_param_error)
    
df_lin_param_errors = pd.DataFrame(lin_param_errors)
df_lin_param_errors

,iteration,train_rmse,test_rmse
0,0,971.413869,972.428043
1,1,702.885724,704.200329
2,2,586.607775,587.057298
3,3,537.883314,537.169014
4,4,516.386797,514.678308
...,...,...,...
655,655,373.896541,366.161219
656,656,373.896350,366.161206
657,657,373.896162,366.161192
658,658,373.895974,366.161180


In [56]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = df_lin_param_errors['iteration'], y = df_lin_param_errors['train_rmse'], 
                         mode = 'lines+markers', name = 'Train RMSE'))
fig.add_trace(go.Scatter(x = df_lin_param_errors['iteration'], y = df_lin_param_errors['test_rmse'], 
                         mode = 'lines+markers', name = 'Test RMSE'))
fig.update_layout(title = 'Train and Test RMSE at various Iterations (Learning Rate = 0.3, Threshold = 0.0001)', 
                  xaxis_title = 'Iteration', yaxis_title = 'RMSE',
                  hovermode = 'x unified')

fig.show()

In [57]:
#Train and Test RMSE for Model built using All Features

print('Training RMSE (All variables): {:.2f}'
      .format(rmse(np.dot(Xf_train.values, lin_select_theta), y_train.values)))
print('Test RMSE (All variables): {:.2f}'
      .format(rmse(np.dot(Xf_test.values, lin_select_theta), y_test.values)))

Training RMSE (All variables): 373.90
Test RMSE (All variables): 366.16


In [58]:
#Train and Test RMSE for Model built using Randomly Sampled Features

alpha = 3e-1
threshold = 1e-4

theta = np.zeros((Xr_train.shape[1], 1), dtype = float)
J_rand_theta = gradientDescentLinear(Xr_train.values, y_train.values, theta, alpha, threshold)

print('Training RMSE (8 random variables): {:.2f}'
      .format(rmse(np.dot(Xr_train.values, J_rand_theta[-1]['theta']), y_train.values)))
print('Test RMSE (8 random variables): {:.2f}'
      .format(rmse(np.dot(Xr_test.values, J_rand_theta[-1]['theta']), y_test.values)))

Training RMSE (8 random variables): 381.58
Test RMSE (8 random variables): 378.66


In [59]:
#Train and Test RMSE for Model built using Important Features

alpha = 3e-1
threshold = 1e-4

theta = np.zeros((Xt_train.shape[1], 1), dtype = float)
J_top_theta = gradientDescentLinear(Xt_train.values, y_train.values, theta, alpha, threshold)

print('Training RMSE (8 important variables): {:.2f}'
      .format(rmse(np.dot(Xt_train.values, J_top_theta[-1]['theta']), y_train.values)))
print('Test RMSE (8 important variables): {:.2f}'
      .format(rmse(np.dot(Xt_test.values, J_top_theta[-1]['theta']), y_test.values)))

Training RMSE (8 important variables): 381.52
Test RMSE (8 important variables): 373.26
